In [1]:
import subprocess
import os
import numpy as np
import meshplot as mp
import igl
import nxmetis
import networkx as nx

In [2]:
def mesh_partitioning(mesh, partitions,partitions_list):
    
    vertices = mesh[0]
    faces = mesh[1]

    submeshes = []
    mapping = []
    


    pb = np.array(partitions_list)

        
    
    mp.plot(mesh[0],mesh[1],pb)

    for i in range(partitions):
        faceList = []
        verticeList = []
        indexes = np.zeros(len(vertices)) - 1
        for j, partition in enumerate(partitions_list):
            if int(partition) == i:
                index1 = indexes[faces[j][0]]
                index2 = indexes[faces[j][1]]
                index3 = indexes[faces[j][2]]

                if index1 == -1 :
                    verticeList.append(vertices[faces[j][0]])
                    index1 = len(verticeList) - 1
                    indexes[faces[j][0]] = index1
                if index2 == -1 :
                    verticeList.append(vertices[faces[j][1]])
                    index2 = len(verticeList) - 1
                    indexes[faces[j][1]] = index2
                if index3 == -1 :
                    verticeList.append(vertices[faces[j][2]])
                    index3 = len(verticeList) - 1
                    indexes[faces[j][2]] = index3

                faceList.append([int(index1), int(index2), int(index3)])
          
        mapping.append(indexes)
        submeshes.append([np.array(verticeList), np.array(faceList)])
    print(len(submeshes))

    return submeshes, mapping

In [ ]:
def partition(mesh,partitions):
    ok = igl.adjacency_list(mesh[1])
    G = nx.Graph()
    for i in range(len(ok)):
        
        G.add_node(i)
        for j in ok[i]:
            if(G.has_edge(i, j) != True):
                G.add_edge(i,j)
    

    [cost, parts] = nxmetis.partition(G, nparts=partitions, recursive=False)
    
    partitions_list = np.zeros(len(mesh[0])) - 1
    
    for i in range(len(parts)):
        for j in range(len(parts[i])):
            partitions_list[parts[i][j]] = i 
    
    flist = []
    for i in mesh[1]:
        if partitions_list[i[0]] == partitions_list[i[1]] == partitions_list[i[2]]:
            flist.append(partitions_list[i[0]])
        else:
            flist.append(-1)
 
        
    return flist